In [ ]:
import scipy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm.auto import tqdm

In [ ]:
N = 2000
m = 2
e = pow(10, -3)
F_param = -2.8
delta_t = 1
n = 10

In [ ]:
def calc_activity(N):
    return pow(np.random.uniform(
        pow(e,F_param), pow(1,F_param), size=N
    ), 1/F_param)*n

In [ ]:
activity_level_of_nodes = calc_activity(N)

In [ ]:
activity_level_of_nodes = calc_activity(N)
res_agg = []
for i in tqdm(range(10000)):
    activity_level_of_nodes = calc_activity(N)
    res_tmp, res2 = np.histogram(
        activity_level_of_nodes, bins = np.linspace(e, 1, num = 50)
    )  
    res_agg.append(res_tmp)
res_agg = np.array(res_agg).mean(axis = 0)
res_agg = res_agg/res_agg.sum()

plt.xlim(e, 1)
# plt.xscale('log')
plt.yscale('log')
plt.xlabel('x')
plt.ylabel('P(x)')

plt.plot(res2[:-1], res_agg)

plt.show()


In [ ]:
def sis_init(start_I_population_size = 5):
    S = 0
    I = 1
    population = np.zeros(N).astype(int)
    population[np.random.choice(N, start_I_population_size, False)] = 1
    return population

In [ ]:
def adn_step(N, activity_level_of_nodes, m):
    active_nodes = (
        activity_level_of_nodes * delta_t > np.random.uniform(0, 1, N)
    )
    active_nodes = np.nonzero(active_nodes)[0]
    edge_candidates = np.repeat(active_nodes, m)
    edges = np.stack(
        [edge_candidates, np.random.choice(range(N), len(edge_candidates))]
    )
    conflicts = edges[0] == edges[1]
    while conflicts.sum():
        edges[1][conflicts] = np.random.choice(range(N), conflicts.sum())
        conflicts = edges[0] == edges[1]
    edges = edges.transpose()
    unique_edges = np.unique(np.sort(edges, axis=1), axis = 1)
    
    return unique_edges

In [ ]:
def sis_step(B, u, population, edges):
    get_state_of_connected_people = np.vectorize(lambda x: population[x])

    state_of_connected_people = np.sum(
        get_state_of_connected_people(edges), axis = 1
    )
    SI_pairs = edges[np.where(state_of_connected_people==1)]
    SI_pairs_to_change = SI_pairs[np.random.uniform(size=len(SI_pairs)) < B]
    
    
    population[population == 1] = (
        (np.random.uniform(size=population.sum()) > u) * 1
    )
    
    population[SI_pairs_to_change.flatten()] = 1

## ADN simulation

In [ ]:
for K in [1, 10, 20]:
    cum_res = []

    for index_of_try in tqdm(range(1000)):
        activity_level_of_nodes = calc_activity(N)
        res = adn_step(N, activity_level_of_nodes, m)
        for i in tqdm(range(K-1), leave=False):
            res = np.concatenate([res, adn_step(N, activity_level_of_nodes, m)])

        res = np.unique(res, axis = 1)
        _, node_degrees = np.unique(res.flatten(), return_counts=True)

        cum_res.extend(node_degrees)

    sizes, counts = np.unique(cum_res, return_counts = True)

    plt.xlim(1, 100)
    plt.ylim(10**(-6), 1)

    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('k')
    plt.ylabel('P(k)')
    plt.plot(sizes, counts/len(cum_res))
    plt.scatter(sizes, counts/len(cum_res), color='r')

    plt.show()

## SIS simulation

In [ ]:
def SIS_full_simulate(B, u, start_population, m, N, max_time, plt_ax, cumulative = False, multiplicate = 1):
    res = []
    for _ in tqdm(range(multiplicate)):
        activity_level_of_nodes = calc_activity(N)
        population = sis_init(start_population)
        infected_count = []
        cumulative_edges = adn_step(N, activity_level_of_nodes, m)
        for _ in tqdm(range(max_time-1), leave=False):
            sis_step(B, u, population, cumulative_edges)
            infected_count.append(population.sum())
            edges = adn_step(N, activity_level_of_nodes, m)
            if cumulative:
                cumulative_edges = np.unique(np.concatenate([cumulative_edges, edges]), axis = 1)
            else:
                cumulative_edges = edges

        res.append(np.array(infected_count))
    res = np.mean(np.array(res), axis = 0)

    plt_ax.set_title('B = ' + str(B) + ' u = ' + str(np.round(u, 2)))
    plt_ax.set_xlabel('time')
    plt_ax.set_ylabel('infected %')
    plt_ax.plot(np.array(infected_count)/N)
    return infected_count

In [ ]:
N = 2000
start_i = int(N * 0.1)
m = 5
max_time = 1000
cumulative = False

B = [0.1, 0.5, 0.9]
u = [1/7, 1/14, 1/31]

multiplicate = 100

fig, axs = plt.subplots(len(B), len(u), figsize=(20, 20))


for i_b, tmp_b in enumerate(B):
    for i_u, tmp_u in enumerate(u):
        SIS_full_simulate(
            B = tmp_b, u = tmp_u, N = N, 
            start_population = start_i, m = m, max_time = max_time, 
            cumulative = cumulative,
            plt_ax = axs[i_b, i_u],
            multiplicate=multiplicate
        )
        
fig.show()

In [ ]:
N = 2000
start_i = int(N * 0.1)
m = 5
max_time = 150
cumulative = True

B = [0.1, 0.5, 0.9]
u = [1/7, 1/14, 1/31]

multiplicate = 5

fig, axs = plt.subplots(len(B), len(u), figsize=(20, 20))


for i_b, tmp_b in enumerate(B):
    for i_u, tmp_u in enumerate(u):
        SIS_full_simulate(
            B = tmp_b, u = tmp_u, N = N, 
            start_population = start_i, m = m, max_time = max_time, 
            cumulative = cumulative,
            plt_ax = axs[i_b, i_u],
            multiplicate=multiplicate
        )
        
fig.show()

In [ ]:
N = 20000
start_i = int(N * 0.1)
m = 5
max_time = 600
cumulative = False

B = [0.1, 0.5, 0.9]
u = [1/7, 1/14, 1/31]

multiplicate = 100

fig, axs = plt.subplots(len(B), len(u), figsize=(20, 20))


for i_b, tmp_b in enumerate(B):
    for i_u, tmp_u in enumerate(u):
        SIS_full_simulate(
            B = tmp_b, u = tmp_u, N = N, 
            start_population = start_i, m = m, max_time = max_time, 
            cumulative = cumulative,
            plt_ax = axs[i_b, i_u],
            multiplicate=multiplicate
        )
        
fig.show()

In [ ]:
N = 20000
start_i = int(N * 0.1)
m = 5
max_time = 150
cumulative = True

B = [0.1, 0.5, 0.9]
u = [1/7, 1/14, 1/31]

multiplicate = 5

fig, axs = plt.subplots(len(B), len(u), figsize=(20, 20))


for i_b, tmp_b in enumerate(B):
    for i_u, tmp_u in enumerate(u):
        SIS_full_simulate(
            B = tmp_b, u = tmp_u, N = N, 
            start_population = start_i, m = m, max_time = max_time, 
            cumulative = cumulative,
            plt_ax = axs[i_b, i_u],
            multiplicate=multiplicate
        )
        
fig.show()